# 5. Issue Selection

* 싸이지먼트 / FinAlgML : 파트 1 - Quantitative Technical Analysis [1]
* 김무성

# Contents

* Market Research
* Risk and Profit Potential
    - Simulation Outline
    - Sidebar - Calculating CAR
    - Drawdown as a Function of Holding Period
    - Profit Potential
    - Risk in Being Short
    - What the Prospector Found
    - Which Issues are "Best"
    - Universal Objective Function
    - Holding Longer
    - Portfolios
* Summary
* Estmating Profit Potential
    - A Monte Carlo Simulation Program
* Program Listing

<img src="figures/cap5.1.png" width=600 />

# Market Research

# Risk and Profit Potential

* Simulation Outline
* Sidebar - Calculating CAR
* Drawdown as a Function of Holding Period
* Profit Potential
* Risk in Being Short
* What the Prospector Found
* Which Issues are "Best"
* Universal Objective Function
* Holding Longer
* Portfolios

## Simulation Outline

<img src="figures/cap5.2.png" width=600 />

<img src="figures/cap5.3.png" width=600 />

## Sidebar - Calculating CAR

* The simulation continues

### The simulation continues

<img src="figures/cap5.4.png" width=600 />

<img src="figures/cap5.5.png" width=600 />

## Drawdown as a Function of Holding Period

<img src="figures/cap5.6.png" width=600 />

## Profit Potential

<img src="figures/cap5.7.png" width=600 />

<img src="figures/cap5.8.png" width=600 />

## Risk in Being Short

<img src="figures/cap5.9.png" width=600 />

## What the Prospector Found

<img src="figures/cap5.10.png" width=600 />

<img src="figures/cap5.10b.png" width=600 />

<img src="figures/cap5.10c.png" width=600 />

<img src="figures/cap5.10d.png" width=600 />

<img src="figures/cap5.10e.png" width=600 />

## Which Issues are "Best"

## Universal Objective Function

## Holding Longer

<img src="figures/cap5.11.png" width=600 />

<img src="figures/cap5.12.png" width=600 />

<img src="figures/cap5.13.png" width=600 />

<img src="figures/cap5.14.png" width=600 />

## Portfolios

# Summary

# Estmating Profit Potential

* A Monte Carlo Simulation Program

## A Monte Carlo Simulation Program

# Program Listing

# 참고자료 